In [1]:
import cv2
import numpy as np




In [3]:
def rotate_yolo_annotations(image, annotations, angle):
    # Get image center
    height, width = image.shape[:2]
    center = (width / 2, height / 2)
    
    # Rotate image
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

    # Rotate annotations
    for i, annotation in enumerate(annotations):
        x_center, y_center, box_width, box_height = annotation

        # Convert YOLO format to OpenCV format (x_top_left, y_top_left, x_bottom_right, y_bottom_right)
        x_top_left = int(x_center - box_width / 2)
        y_top_left = int(y_center - box_height / 2)
        x_bottom_right = int(x_center + box_width / 2)
        y_bottom_right = int(y_center + box_height / 2)

        # Rotate bounding box coordinates
        rotated_points = cv2.transform(np.array([[[x_top_left, y_top_left], [x_bottom_right, y_bottom_right]]]), rotation_matrix)
        x_top_left_rotated, y_top_left_rotated = rotated_points[0][0]
        x_bottom_right_rotated, y_bottom_right_rotated = rotated_points[0][1]

        # Calculate new rotated coordinates
        new_x_center = (x_top_left_rotated + x_bottom_right_rotated) / 2
        new_y_center = (y_top_left_rotated + y_bottom_right_rotated) / 2
        new_box_width = abs(x_bottom_right_rotated - x_top_left_rotated)
        new_box_height = abs(y_bottom_right_rotated - y_top_left_rotated)

        # Update annotations
        annotations[i] = (new_x_center, new_y_center, new_box_width, new_box_height)

    return rotated_image, annotations

# Example usage
image_path = "Template9_Instance6.jpg"
annotations = [(0.15722352941176468, 0.7840734839476813, 0.30772436974789913 , 0.09988109393579059)]  # Example YOLO format annotation
angle = 180  # Angle by which to rotate the image and annotations

# Read image
image = cv2.imread(image_path)

# Rotate annotations
rotated_image, rotated_annotations = rotate_yolo_annotations(image, annotations, angle)

# Display rotated image
cv2.imshow("Rotated Image", rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Rotated Annotations:", rotated_annotations)

Rotated Annotations: [(595.0, 841.0, 0, 0)]


In [4]:
def rotate_yolo_annotations(annotations, image_height, angle):
    rotated_annotations = []
    angle_rad = np.radians(angle)
    for annotation in annotations:
        x_center, y_center, box_width, box_height = annotation

        # Convert YOLO format to image coordinates
        x_top_left = (x_center - box_width / 2) * image_height
        y_top_left = (y_center - box_height / 2) * image_height
        x_bottom_right = (x_center + box_width / 2) * image_height
        y_bottom_right = (y_center + box_height / 2) * image_height

        # Calculate rotated coordinates
        x_center_rotated = x_center
        y_center_rotated = y_center
        box_width_rotated = box_width
        box_height_rotated = box_height

        # Apply rotation if angle is not 0
        if angle != 0:
            x_center_rotated = x_center
            y_center_rotated = y_center
            box_width_rotated = box_width
            box_height_rotated = box_height

            # Rotate the box around the image center
            x_center_rotated, y_center_rotated = rotate_point(x_center, y_center, 0.5, 0.5, angle_rad)
            x_top_left_rotated, y_top_left_rotated = rotate_point(x_top_left, y_top_left, 0, 0, angle_rad)
            x_bottom_right_rotated, y_bottom_right_rotated = rotate_point(x_bottom_right, y_bottom_right, 1, 1, angle_rad)

            # Convert back to YOLO format
            box_width_rotated = (x_bottom_right_rotated - x_top_left_rotated) / image_height
            box_height_rotated = (y_bottom_right_rotated - y_top_left_rotated) / image_height
            x_center_rotated = (x_top_left_rotated + box_width_rotated / 2) / image_height
            y_center_rotated = (y_top_left_rotated + box_height_rotated / 2) / image_height

        rotated_annotations.append((x_center_rotated, y_center_rotated, box_width_rotated, box_height_rotated))

    return rotated_annotations

def rotate_point(x, y, cx, cy, angle):
    # Translate point to origin
    x -= cx
    y -= cy

    # Apply rotation
    x_new = x * np.cos(angle) - y * np.sin(angle)
    y_new = x * np.sin(angle) + y * np.cos(angle)

    # Translate point back
    x_new += cx
    y_new += cy

    return x_new, y_new

# Example usage
annotations = [(0.15722352941176468, 0.7840734839476813, 0.30772436974789913 , 0.09988109393579059)]  # Example YOLO format annotation
image_height = 841  # Example image height in pixels
angle = 180  # Angle by which to rotate the annotations

# Rotate annotations
rotated_annotations = rotate_yolo_annotations(annotations, image_height, angle)

print("Rotated Annotations:", rotated_annotations)

Rotated Annotations: [(-0.003542882140944641, -0.7341909054534196, -0.30534624846371367, -0.0975029726516051)]


In [5]:
import os
import json
import torch
from PIL import Image, ImageDraw
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import tkinter as tk
from tkinter import ttk

In [6]:
image_folder = 'D:/Revert Annotations/images'
annotation_folder = 'D:/Revert Annotations/annotations'
output_folder = 'D:/Revert Annotations/revert_annotations'


In [43]:
for filename in os.listdir(annotation_folder):
    if filename.endswith('.json'):
        with open(os.path.join(annotation_folder, filename) ,  'r', encoding='utf-8') as f:
            annotations[filename] = json.load(f)
            for annotation_file, annotation_data in annotations.items():
                for label_data in annotation_data:
                        if 'bbox' in label_data:
                            bbox = label_data['bbox']
                            # Swap y-coordinates to correct inversion
                            bbox = [(x, 841 - y) for x, y in bbox]
                            # Ensure correct ordering of coordinates (x0, y0, x1, y1)
                            x_min = min(bbox[0][0], bbox[1][0])
                            y_min = min(bbox[0][1], bbox[1][1])
                            x_max = max(bbox[0][0], bbox[1][0])
                            y_max = max(bbox[0][1], bbox[1][1])
                            bbox_ordered = [(x_min, y_min), (x_max, y_max)]
                            bbox = bbox_ordered

                            with open(os.path.join(annotation_folder, filename), 'w', encoding='utf-8') as json_file:
                                json.dump(bbox, json_file)
            
        
    

TypeError: string indices must be integers

In [48]:
def load_annotations(annotation_folder):
    annotations = {}
    for filename in os.listdir(annotation_folder):
        if filename.endswith('.json'):
            with open(os.path.join(annotation_folder, filename)) as f:
                annotations[filename] = json.load(f)
                for annotation_file, annotation_data in annotations.items():
                    # print("the annoations file in first is " , annotation_file)
                    # print("the annoations data in first is " , annotation_data)
                    for label, label_data in annotation_data.items():
                        if 'bbox' in label_data:
                            bbox = label_data['bbox']
                            # Swap y-coordinates to correct inversion
                            bbox = [(x, 841 - y) for x, y in bbox]
                            # Ensure correct ordering of coordinates (x0, y0, x1, y1)
                            x_min = min(bbox[0][0], bbox[1][0])
                            y_min = min(bbox[0][1], bbox[1][1])
                            x_max = max(bbox[0][0], bbox[1][0])
                            y_max = max(bbox[0][1], bbox[1][1])
                            bbox_ordered = [(x_min, y_min), (x_max, y_max)]
                            bbox = bbox_ordered
                            # print("The bbox here is :",bbox)
                            # json.dump(bbox,f)
                            with open(os.path.join(annotation_folder, filename), 'w', encoding='utf-8') as json_file:
                                json.dump(bbox, json_file)
                    
                    
                
                
    return annotations

In [49]:
def visualize_image(image_path, annotation_data, output_folder):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    for label, label_data in annotation_data.items():
        if 'bbox' in label_data:
            bbox = label_data['bbox']
            # Swap y-coordinates to correct inversion
            bbox = [(x, image.height - y) for x, y in bbox]
            # Ensure correct ordering of coordinates (x0, y0, x1, y1)
            x_min = min(bbox[0][0], bbox[1][0])
            y_min = min(bbox[0][1], bbox[1][1])
            x_max = max(bbox[0][0], bbox[1][0])
            y_max = max(bbox[0][1], bbox[1][1])
            bbox_ordered = [(x_min, y_min), (x_max, y_max)]
            bbox = bbox_ordered
            # print("the inverterd annoations are : " , bbox_ordered)
            # print("The new bbox condinates are : ", bbox)
            draw.rectangle(bbox_ordered, outline='red')
    output_filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, output_filename)
    image.save(output_path)
    print(f"Image with annotations saved successfully: {output_path}")


In [50]:
def visualize_annotations(image_folder, annotation_folder, output_folder):
    annotations = load_annotations(annotation_folder)
    total_files = len(annotations)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    with ThreadPoolExecutor() as executor:
        for annotation_file, annotation_data in tqdm(annotations.items(), total=total_files, desc="Processing images"):
            image_filename = annotation_file.replace('.json', '.jpg')
            # print(" ------------------------- ")
            # print("the annotations data is :  ",annotation_data)
            image_path = os.path.join(image_folder, image_filename)
            if not os.path.exists(image_path):
                print(f"Image {image_filename} not found.")
                continue
            executor.submit(visualize_image, image_path, annotation_data, output_folder)


In [51]:
visualize_annotations(image_folder, annotation_folder,output_folder)


Processing images: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

Image Template9_Instance6_sample.jpg not found.
